# Задание 1


Выберите 5 языков в википедии (не тех, что использовались в семинаре). Скачайте по 10 случайных статей для каждого языка. Предобработайте тексты, удаляя лишние теги/отступы/разделители (если они есть). Разделите тексты на предложения и создайте датасет, в котором каждому предложению соответствует язык. Кластеризуйте тексты, используя эбмединг модель из прошлого семинара и любой алгоритм кластеризации. Проверьте качество кластеризации с помощь метрики ARI. Отдельно проанализируйте 3 ошибочно кластеризованных текста (если такие есть).

In [19]:
import wikipedia
import numpy as np

In [20]:
def get_random_page():
    try:
        return wikipedia.random(1)
    except Exception:
        return get_random_page()
    
def get_page_by_name(name:str):
    try:
        return wikipedia.page(name)
    except wikipedia.DisambiguationError as e:
        random_option = np.random.choice(e.options)
        return get_page_by_name(random_option)


####  Код ниже можнно не запускать, под ним ячейка со спиклованным объектом

In [29]:
lang_to_pages = {
    "zh": dict(),  #  китайский
    "ja": dict(),  #  японский
    "ko": dict(),  #  корейский
    "hi": dict(),  #  хинди
    "lv": dict(),  #  латышский
}
for lang in lang_to_pages:
    wikipedia.set_lang(lang)
    for _ in range(10):
        random_page = get_random_page()
        lang_to_pages[lang][random_page] = get_page_by_name(random_page).content

d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [34]:
import pickle

with open(r"data/lang_to_pages.pkl", "wb") as file:
    pickle.dump(lang_to_pages, file)

## Загрузка объекта

In [40]:
with open(r"data/lang_to_pages_COPY.pkl", "rb") as file:
    lang_to_pages = pickle.load(file)


1324

# Задание 2

Загрузите корпус `annot.opcorpora.no_ambig_strict.xml.bz2` с OpenCorpora. Найдите в корпусе самые частотные морфологически омонимичные словоформы (те, которым соответствует разный грамматический разбор в разных предложениях). Также найдите словоформы с самых большим количеством вариантов грамматических разборов.

## Задание 3
Загрузите один и з файлов корпуса Syntagrus - https://github.com/UniversalDependencies/UD_Russian-SynTagRus/tree/master (можно взять тестовый)

Преобразуйте все разборы предложений в графовые структуры через DependencyGraph, выберите 3 любых отношения и для каждого найдите топ-5 самых встречаемых пар слов, связанных этим отношением. 

Для самой частотной пары слов в каждом из отношений вытащите все подзависимые слова для каждого из них во всех предложениях (используя `flatten(get_subtree(d.nodes, index_of_a_word)` и сортируя результат по порядку слов в предложениях, аналогично тому как я делал с summaries только у вас будет два слова) 
В итоге у вас должен получится что-то такое:

```
### отношение
relation_name

### топ 5 пар слов связанных этим отношением
(word1, word2), (word3, word4), (word5, word6), (word7, word8), (word9, word10)

### подзависимые для самого частотного
(subword word1 subword, word2 subword subword)

... (и так три раза)
```
